<a href="https://colab.research.google.com/github/CrisMcode111/DI_Bootcamp/blob/main/Exercises_XP_MCP_Student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercises XP: Minimal MCP over STDIO (Student)

Build a tiny MCP server and client that talk over STDIO. This code is supposed to be executed in a local jupyter notebook not Colab's notebook.

## What you'll learn
- How MCP structures hosts/clients/servers and why STDIO is great locally.
- How to register a tool (action) and a resource (read-only context) on a server.
- How to write a client that initializes, lists, and invokes those features.

## Setup
Run the install cell, then restart the runtime if Colab asks. Python 3.10+ required.

In [1]:
# Install MCP CLI + SDK
%pip install -qU "mcp[cli]"

In [ ]:
# Quick verify
!python --version
!mcp --help | head -n 5

## A. Server (server.py)
Create a small MCP server named "Demo" with:
- Tool `add(a: int, b: int) -> int` returning the sum.
- Resource template `greeting://{name}` returning "Hello, {name}!".
- Start the STDIO loop in `__main__`.

In [ ]:
%%writefile server.py
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("Demo")

@mcp.tool()
def add(a: int, b: int) -> int:
    """Return the sum of two integers."""
    # TODO: return the sum
    ...

@mcp.resource("greeting://{name}")
def greet(name: str) -> str:
    """Return a greeting for the given name."""
    # TODO: return "Hello, {name}!"
    ...

if __name__ == "__main__":
    # TODO: start the server loop over STDIO
    ...

## B. Client (client.py)
Write a client that:
1) Spawns the server via STDIO using the MCP CLI.
2) Initializes a session.
3) Lists resources and tools, printing their names.
4) Reads `greeting://hello` and prints it.
5) Calls tool `add` with a=1, b=7 and prints the result.

In [ ]:
%%writefile client.py
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

server_params = StdioServerParameters(command="mcp", args=["run", "server.py"], env=None)


def extract_content(payload):
    """Best-effort to pull text from MCP responses."""
    if hasattr(payload, "contents"):
        contents = payload.contents
        if contents:
            first = contents[0]
            if hasattr(first, "text"):
                return first.text
            if isinstance(first, dict) and "text" in first:
                return first["text"]
            return str(first)
    if hasattr(payload, "content"):
        return payload.content
    return str(payload)


async def run():
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()

            # TODO: list resources and print their URIs
            # TODO: list tools and print their names

            # TODO: read greeting://hello and print the content

            # TODO: call add with a=1, b=7 and print the result


if __name__ == "__main__":
    asyncio.run(run())

## C. Run
One terminal (client spawns server):
```
python client.py
```

Or two terminals:
```
mcp run server.py
python client.py
```

In Colab, run the next cell (client will spawn the server automatically).

In [ ]:
# Run the client (spawns the server over STDIO)
!python client.py

## Troubleshooting
- `mcp: command not found` ? rerun the install cell or restart runtime.
- Connection closed ? open a second terminal and run `mcp run server.py` to check server errors.
- Type errors ? ensure JSON args are ints for `add`.